In [14]:
from ucimlrepo import fetch_ucirepo
import pandas as pd
import numpy as np

# fetch dataset
adult = fetch_ucirepo(id=2)

# data (as pandas dataframes)
X = adult.data.features
y = adult.data.targets

# metadata
print(adult.metadata)

# variable information
print(adult.variables)




{'uci_id': 2, 'name': 'Adult', 'repository_url': 'https://archive.ics.uci.edu/dataset/2/adult', 'data_url': 'https://archive.ics.uci.edu/static/public/2/data.csv', 'abstract': 'Predict whether annual income of an individual exceeds $50K/yr based on census data. Also known as "Census Income" dataset. ', 'area': 'Social Science', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 48842, 'num_features': 14, 'feature_types': ['Categorical', 'Integer'], 'demographics': ['Age', 'Income', 'Education Level', 'Other', 'Race', 'Sex'], 'target_col': ['income'], 'index_col': None, 'has_missing_values': 'yes', 'missing_values_symbol': 'NaN', 'year_of_dataset_creation': 1996, 'last_updated': 'Tue Sep 24 2024', 'dataset_doi': '10.24432/C5XW20', 'creators': ['Barry Becker', 'Ronny Kohavi'], 'intro_paper': None, 'additional_info': {'summary': "Extraction was done by Barry Becker from the 1994 Census database.  A set of reasonably clean records was extracted using the fol

In [15]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

#80-20 random split
rng = np.random.default_rng(0)
idx = np.arange(len(X))
rng.shuffle(idx)
split = int(0.8 * len(idx))

train_idx = idx[:split]
test_idx  = idx[split:]

X_train = X.iloc[train_idx].to_numpy()
y_train = y.iloc[train_idx].to_numpy()

X_test = X.iloc[test_idx].to_numpy()
y_test = y.iloc[test_idx].to_numpy()

num_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

cat_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder())
])

#Transform Data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_pipe, ['age', 'capital-gain', 'capital-loss']),  # Apply StandardScaler to numerical columns
        ('cat', cat_pipe, ['workclass', 'education', 'martial-status', 'ocupation', 'relationship', 'race', 'sex', 'native-country', 'education' ]) # Apply OneHotEncoder to categorical columns
    ],
    remainder='passthrough' # Keep other columns
)

X_transformed = preprocessor.fit_transform(X)

ValueError: A given column is not a column of the dataframe